<a href="https://colab.research.google.com/github/yeqinghuang516/transfer-learning-and-inference-acceleration/blob/master/ResNet_152_Train_on_Cars_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook is RECOMMENDED to run on Google Colab

## Copy our Project Repository from Github

In [0]:
! git clone https://github.com/yeqinghuang516/transfer-learning-and-inference-acceleration.git
% cd transfer-learning-and-inference-acceleration

## Download the Training Data

In [0]:
!wget http://imagenet.stanford.edu/internal/car196/cars_train.tgz
!wget https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
!tar -xvf cars_train.tgz
!tar -xvf car_devkit.tgz

In [0]:
%matplotlib notebook
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
import torch.utils.data as td
import torchvision as tv
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
import scipy.io
import nntools as nt

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Define Dataset Class

In [0]:
class CarDataset(td.Dataset):
    def __init__(self, root_dir, mode = 'train', image_size = (224,224), train = False):
        super(CarDataset, self).__init__()
        self.image_size = image_size
        self.mode = mode
        self.cars_annos = scipy.io.loadmat(root_dir + 'devkit/cars_' + mode + '_annos')
        self.annos = self.cars_annos['annotations']
        self.annos = np.transpose(self.annos)
        self.images_dir = root_dir + 'cars_' + mode + '/'
        self.train = train
        
    def __len__(self):
        return self.annos.shape[0]
    
    def __getitem__(self, idx):
        img_path = self.images_dir + "%05d.jpg" % (idx + 1)
        bbox_x1 = self.annos[idx][0][0][0][0]
        bbox_y1 = self.annos[idx][0][1][0][0]
        bbox_x2 = self.annos[idx][0][2][0][0]
        bbox_y2 = self.annos[idx][0][3][0][0]
        d = self.annos[idx][0][4][0][0]
        d = int(d)
        img = Image.open(img_path).convert('RGB')
        img = img.crop([max(0 , bbox_x1 - 16), max(0, bbox_y1 - 16), min(img.size[0], bbox_x2 + 16), min(img.size[1], bbox_y2 + 16)])
        h, w = img.size
        dim_diff = np.abs(h - w)
        pad1, pad2 = dim_diff // 2, dim_diff - dim_diff // 2
        pad = (0, pad1, 0, pad2) if h >= w else (pad1, 0, pad2, 0)
        data_aug = [
            tv.transforms.RandomHorizontalFlip(),
            tv.transforms.RandomAffine(10),
            tv.transforms.RandomRotation(10),
                   ]
        
        transform_list = [
            tv.transforms.Pad(pad, fill = 0, padding_mode = 'constant'), 
            tv.transforms.Resize(self.image_size),
            tv.transforms.ToTensor(),
            tv.transforms.Normalize(mean = [0.5, 0.5 ,0.5], std = [0.5, 0.5 ,0.5]),
        ]
        
        if self.train:
          transform_list.insert(2, tv.transforms.ColorJitter(brightness= 0.5, contrast= 0.5, saturation= 0.5, hue=0.5))
        
        if self.train and np.random.random() < 0.5:
          transform_list.insert(2, tv.transforms.RandomApply(data_aug))        
        
        transform = tv.transforms.Compose(transform_list)
        
        x = transform(img)
        return x, (d - 1)
    
    def number_of_classes(self):
        classes = []
        for annotations in self.annos:
            classes.append(annotations[0][4][0][0])
        classes = np.array(classes)
        return classes.max()

## Initiate Training and Validation Dataset

In [0]:
dataset_root_dir = '/content/transfer-learning-and-inference-acceleration/'
dataset = CarDataset(dataset_root_dir, train = True)

full_length = len(dataset)
train_size = int(0.7 * full_length)
test_size = full_length - train_size
idx = np.random.permutation(range(full_length))
train_idx = idx[:train_size]
test_idx = idx[train_size: ]

trainset = td.Subset(dataset, train_idx)
dataset = CarDataset(dataset_root_dir)
testset = td.Subset(dataset, test_idx)

## Define the Transferred Network

In [0]:
class NNClassifier(nt.NeuralNetwork):
    def __init__(self):
        super(NNClassifier, self).__init__()
        self.cross_entropy = nn.CrossEntropyLoss()
        
    def criterion(self, y, d):
        return self.cross_entropy(y, d)

In [0]:
class ResNet152Transfer(NNClassifier):
    
    def __init__(self, num_classes, fine_tuning=True):
        super(ResNet152Transfer, self).__init__()
        resnet152 = tv.models.resnet152(pretrained= True)
        for param in resnet152.parameters():
            param.requires_grad = fine_tuning
                
        self.model = resnet152
        self.model.fc = nn.Linear(2048, num_classes)
        
    def forward(self, x):
      y = self.model(x)
      return y

## Define StatsManager for Recording Loss and Accuracy

In [0]:
class ClassificationStatsManager(nt.StatsManager):
    def __init__(self):
        super(ClassificationStatsManager, self).__init__()
        
    def init(self):
        super(ClassificationStatsManager, self).init()
        self.running_accuracy = 0
        
    def accumulate(self, loss, x, y, d):
        super(ClassificationStatsManager, self).accumulate(loss, x, y, d)
        _, l = torch.max(y, 1)
        _, l_top5 = torch.topk(y, 5)

        self.running_accuracy_top1 += torch.mean((l == d).float())
        self.running_accuracy_top5 += torch.mean((l_top5 == d.view(-1,1).repeat(1,5)).float()) * 5
        
    def summarize(self):
        loss = super(ClassificationStatsManager, self).summarize()
        accuracy_top1 = 100 * self.running_accuracy_top1 / self.number_update
        accuracy_top5 = 100 * self.running_accuracy_top5 / self.number_update
        return {'loss': loss, 'top 1 accuracy': accuracy_top1, 'top 5 accuracy': accuracy_top5}

## Initiate Experiment

In [0]:
lr = 1e-3
net = ResNet152Transfer(dataset.number_of_classes())
net = net.to(device)
adam = torch.optim.Adam(net.parameters(), lr=lr)
stats_manager = ClassificationStatsManager()
exp1 = nt.Experiment(net, trainset, testset, adam, stats_manager,
output_dir="ResNet152", batch_size = 64, perform_validation_during_training=True)

In [0]:
def plot(exp):
  if len(exp.history) is not 0:
    print('training loss: ', exp.history[exp.epoch - 1][0]['loss'])
    print('validation loss: ', exp.history[exp.epoch - 1][1]['loss'])
    print('training accuracy: ', exp.history[exp.epoch - 1][0]['accuracy'])
    print('validation accuracy: ', exp.history[exp.epoch - 1][1]['accuracy'])

## Start Training

In [0]:
exp1.run(num_epochs= 100, plot=lambda exp1: plot(exp1))